## Script to estimate building repair costs and damage severities 

In [22]:
import scipy.stats
import math
import pandas as pd

In [23]:
# Import fragility curve parameters and repair costs
vuln = pd.read_csv('function_params.csv')

# Look at the column names and rename if needed
print(vuln.columns)
new_cols =['Country', 'Building_type', 'mean', 'dispersion', 'tot_rep_cost']

vuln.columns = new_cols

vuln

Index(['Country', 'Building cluster', 'Mean (tephra loading in kPa)',
       'Standard deviation', 'Total replacement cost (USD)'],
      dtype='object')


,Country,Building_type,mean,dispersion,tot_rep_cost
0,Indonesia,Weak,3.25,0.516,15400
1,Indonesia,Strong,7.53,0.529,37800
2,Philippines,Weak,3.60,0.482,9500
3,Philippines,Strong,8.83,0.498,20300
4,Aggregated,Weak,3.40,0.501,12800
5,Aggregated,Strong,7.80,0.522,34100


In [24]:
# Check how to extract p-values from a cdf where the mean (loc) and dispersion (scale) are specified
scipy.stats.norm(loc = math.log(3.25), 
                 scale = 0.516).cdf(math.log(4))

0.656305852888741

In [25]:
# Write a function to return the p-values of a cdf when given a tephra load
# Note that we take the log of the mean tephra load and the tephra load used in the calculation

def damage_ratio(t_load, mean, disp):
    return scipy.stats.norm(loc = math.log(mean), scale = disp).cdf(math.log(t_load))

In [77]:
# Make up a DataFrame to test function on

data= {'tephra_load':[1,2,3,4,5,6,7,8,9,10],
       'n_buildings':[10000, 200, 3000, 40, 500, 60, 7000, 80, 9, 10]}

df = pd.DataFrame(data)
df

,tephra_load,n_buildings
0,1,10000
1,2,200
2,3,3000
3,4,40
4,5,500
5,6,60
6,7,7000
7,8,80
8,9,9
9,10,10


In [78]:
# Calculate damage ratio for each row based on tephra load

# Create an empty column for the DataFrame where values will be stored    
new_column = "dratio"
df[new_column] = None

# Iterate over the rows
for idx, row in df.iterrows():
    
    # Calculate the damage ratio - change the number inside .loc to access different paris of parameters
    drat = damage_ratio(t_load = row['tephra_load'], mean = vuln['mean'].loc[0], disp = vuln['dispersion'].loc[0])
    
    # Update the value of 'Celsius' column with the converted value
    df.at[idx, new_column] = drat
    
df

,tephra_load,n_buildings,dratio
0,1,10000,0.011179
1,2,200,0.173376
2,3,3000,0.438363
3,4,40,0.656306
4,5,500,0.798099
5,6,60,0.88262
6,7,7000,0.931483
7,8,80,0.959569
8,9,9,0.975808
9,10,10,0.985303


In [79]:
# Calculate repair costs for each row 

# Start by accessing the appropriate replacement cost from the 'vuln' dataframe
df['rep_cost_total'] = vuln['tot_rep_cost'].loc[0]

# Multiply the total by the damage ratio and by the number of buildings that were 
# exposed to a specific tephra fall load
df["loss"] = df["dratio"] * df["rep_cost_total"] * df["n_buildings"]

df

,tephra_load,n_buildings,dratio,rep_cost_total,loss
0,1,10000,0.011179,15400,1721634.686089
1,2,200,0.173376,15400,533999.181058
2,3,3000,0.438363,15400,20252359.324985
3,4,40,0.656306,15400,404284.405379
4,5,500,0.798099,15400,6145362.974585
5,6,60,0.88262,15400,815540.991427
6,7,7000,0.931483,15400,100413885.07044
7,8,80,0.959569,15400,1182189.442828
8,9,9,0.975808,15400,135246.925013
9,10,10,0.985303,15400,151736.647083


In [80]:
print('Total repair cost of all eruptions in analysis: $', int(sum(df['loss'])/1000000), 'million USD')


Total repair cost of all eruptions in analysis: $ 131 million USD


## Converting damage ratios back to damage state

In [81]:
# Read in damage ratio ranges (0-1) that correspond to each damage state (0-5)
dr2ds = pd.read_csv('ratio_to_damage.csv')

# Look at the column names and rename if needed
print(dr2ds.columns)
new_cols =['DS', 'DS_level ', 'cdv ', 'dr_range', 'dr_lwr', 'dr_upr']

dr2ds.columns = new_cols

dr2ds

Index(['DS', 'Damage severity level ', 'Central damage value ', 'Range',
       'dr_lwr', 'dr_upr'],
      dtype='object')


,DS,DS_level,cdv,dr_range,dr_lwr,dr_upr
0,0,None,0.005,0-0.01,0.00,0.01
1,1,Light,0.020,0.01-0.05,0.01,0.05
2,2,Moderate,0.100,0.05-0.2,0.05,0.20
3,3,Heavy,0.400,0.2-0.6,0.20,0.60
4,4,Severe,0.750,0.6-0.9,0.60,0.90
5,5,Collapse,1.000,0.9-1.0,0.90,1.00


In [82]:
## Use the range values to specify damage state
# Make an empty list to hold values
damage_states = []

# Specify number of rows for the loop
n = len(df)

for i in range(n):
    
    # damage ratios less than 0.01 will be DS0
    if df['dratio'][i] < dr2ds['dr_lwr'][1]:
        damage_states.append(dr2ds['DS'][0])
        
    # damage ratios >= 0.01 and < 0.05 will be DS1   
    elif df['dratio'][i] >= dr2ds['dr_lwr'][1] and df['dratio'][i] < dr2ds['dr_lwr'][2]:
        damage_states.append(dr2ds['DS'][1])
        
    # damage ratios >= 0.05 and < 0.2 will be DS2    
    elif df['dratio'][i] >= dr2ds['dr_lwr'][2] and df['dratio'][i] < dr2ds['dr_lwr'][3]:
        damage_states.append(dr2ds['DS'][2])
    
    # you get the idea
    elif df['dratio'][i] >= dr2ds['dr_lwr'][3] and df['dratio'][i] < dr2ds['dr_lwr'][4]:
        damage_states.append(dr2ds['DS'][3])
    
    # yep
    elif df['dratio'][i] >= dr2ds['dr_lwr'][4] and df['dratio'][i] < dr2ds['dr_lwr'][5]:
        damage_states.append(dr2ds['DS'][4])
        
    # last one, damage ratios >= 0.9 are DS5
    elif df['dratio'][i] >= dr2ds['dr_lwr'][5]:
        damage_states.append(dr2ds['DS'][5])

In [83]:
# Check it worked by printing the first 10 values in the list
print(damage_states[:10])

[1, 2, 3, 4, 4, 4, 5, 5, 5, 5]


In [87]:
# Attach the list of damage states to the main DataFrame
df = df.assign(DS=damage_states)
df

,tephra_load,n_buildings,dratio,rep_cost_total,loss,DS
0,1,10000,0.011179,15400,1721634.686089,1
1,2,200,0.173376,15400,533999.181058,2
2,3,3000,0.438363,15400,20252359.324985,3
3,4,40,0.656306,15400,404284.405379,4
4,5,500,0.798099,15400,6145362.974585,4
5,6,60,0.88262,15400,815540.991427,4
6,7,7000,0.931483,15400,100413885.07044,5
7,8,80,0.959569,15400,1182189.442828,5
8,9,9,0.975808,15400,135246.925013,5
9,10,10,0.985303,15400,151736.647083,5


In [ ]:
# Calculate the total number of buildings in each damage state
grouped = df.groupby(df['DS'])